In [ ]:
import os

package_path = '/home/meganorm-smkia/Code/MEGaNorm/'
os.chdir(package_path)

from utils.parallel import submit_jobs, check_jobs_status, collect_results


In [ ]:
### Paths and configs

username = 'meganorm-smkia'

data_path = '/project/meganorm/Data/camcan/CamCAN/cc700/meg/pipeline/release005/BIDSsep/derivatives_rest/aa/AA_movecomp_transdef/aamod_meg_maxfilt_00003/'
target_dir = '/home/meganorm-smkia/Data/CamCAN/Features/'

log_path = '/home/meganorm-smkia/temp/log/'
temp_path = '/home/meganorm-smkia/temp/tmp/'

batch_file = '/home/meganorm-smkia/Code/MEGaNorm/src/batch_job.sh'

if not os.path.isdir(log_path):
    os.makedirs(log_path)

In [ ]:
subjects = os.listdir(data_path)
 
# Running Jobs
start_time = submit_jobs(batch_file, data_path, subjects, temp_path)
# Checking jobs
failed_job_names= check_jobs_status(username, start_time)

while len(failed_job_names)>0:
    # Re-running Jobs
    start_time = submit_jobs(batch_file, data_path, failed_job_names, temp_path)
    # Checking jobs
    failed_job_names= check_jobs_status(username, start_time)

collect_results(target_dir, subjects, temp_path, file_name='features')